In [ ]:
!pip install flask pyngrok PyMuPDF

In [ ]:
!pip install openai

In [ ]:
from flask import Flask, request, render_template_string
import fitz  # PyMuPDF
import threading
import webbrowser
import threading
import fitz  # PyMuPDF
import nest_asyncio
nest_asyncio.apply()
import re
import numpy as np
from numpy.linalg import norm
from gensim.models.doc2vec import Doc2Vec
import random

from openai import OpenAI

OPENROUTER_API_KEY = "" #pls do not steal ok


eu_countries = [
    {"name": "Europe", "value": 15},
    {"name": "Austria", "value": 5},
    {"name": "Belgium", "value": 6},
    {"name": "Bulgaria", "value": 4},
    {"name": "Croatia", "value": 5},
    {"name": "Cyprus", "value": 3},
    {"name": "Czech Republic", "value": 6},
    {"name": "Denmark", "value": 7},
    {"name": "Estonia", "value": 4},
    {"name": "Finland", "value": 6},
    {"name": "France", "value": 8},
    {"name": "Germany", "value": 8},
    {"name": "Greece", "value": 4},
    {"name": "Hungary", "value": 5},
    {"name": "Ireland", "value": 6},
    {"name": "Italy", "value": 7},
    {"name": "Latvia", "value": 3},
    {"name": "Lithuania", "value": 4},
    {"name": "Luxembourg", "value": 6},
    {"name": "Malta", "value": 3},
    {"name": "Netherlands", "value": 7},
    {"name": "Poland", "value": 5},
    {"name": "Portugal", "value": 6},
    {"name": "Romania", "value": 4},
    {"name": "Slovakia", "value": 5},
    {"name": "Slovenia", "value": 4},
    {"name": "Spain", "value": 7},
    {"name": "Sweden", "value": 6}
]

def get_country_value(name):
    for country in eu_countries:
        if country["name"] == name:
            return country["value"]
    return 1

def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()
    
    # Remove punctuation from the text
    text = re.sub('[^a-z]', ' ', text)
    
    # Remove numerical values from the text
    text = re.sub(r'\d+', '', text)
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return text

def percentage_to_color(percentage: float):
    # Clamp percentage to [0, 100]
    percentage = max(0, min(percentage, 100))
    
    # Map percentage to hue from red (0°) to green (120°)
    hue = int((percentage / 100) * 120)
    
    return hue

def LLM_ranker(resume, job):
  
  client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= OPENROUTER_API_KEY,
  )
  completion = client.chat.completions.create(
    model= "google/gemini-2.5-flash-preview-05-20", #"openai/gpt-4o", #"deepseek/deepseek-r1-0528:free",
    messages=[
      {
        "role": "user",
        "content": f"This is my resume, and a job description. Return a score from 0 to 100 based on how relevant this job description is. It is a continuum, so grade based on that. Return ONLY a number, nothing else. Be lenient, give hiegher scores, these are students with CV, be kind to them. Feel hope for every resume. Resume: {resume}. Job description: {job}"
        }
    ]
  )
  return completion.choices[0].message.content.strip()

JobDescriptions=[
 {"JobName": "Administrative Assistant", "JobDescription": """Company: CivicEdge Consulting. We are looking for a reliable and organized Administrative Assistant to support daily operations within our office. This role requires strong multitasking abilities and a positive, can-do attitude. Key responsibilities include managing schedules, organizing meetings, handling correspondence, and maintaining internal databases. You will also help coordinate travel arrangements and support documentation needs for various teams. The successful candidate will have strong organizational and communication skills, as well as proficiency in Microsoft Office or Google Workspace tools. Prior experience in an administrative or support role is beneficial but not mandatory. We offer a structured environment, clear workflows, and supportive leadership to help you succeed and grow in the role."""}
,{"JobName": "Customer Support Representative", "JobDescription": """Company: LoopLogic Systems. We are currently hiring a friendly and solution-oriented Customer Support Representative to assist our customers via email, phone, and live chat. This is a great opportunity for someone who enjoys helping others and thrives in a fast-paced environment. Your main responsibilities will include resolving customer inquiries, troubleshooting basic issues, and escalating complex concerns to relevant departments. Clear communication and a calm demeanor are key in delivering excellent support. The ideal candidate is empathetic, tech-savvy, and has strong problem-solving abilities. Prior experience in a customer service role is an advantage, but full training will be provided. You will join a supportive team with opportunities for advancement and skill development. Flexible working hours are available, including remote options. """}
,{"JobName": "Data Entry Clerk", "JobDescription" : """Company: Clearform Solutions. We are looking for a detail-oriented Data Entry Clerk to support our administrative team. The ideal candidate will be highly organized, accurate, and comfortable working with large sets of information. Responsibilities include entering and updating data in company databases, verifying accuracy, and generating reports as needed. You will also assist with basic administrative tasks such as filing and email correspondence. This role requires excellent attention to detail, strong typing skills, and the ability to manage repetitive tasks efficiently. Experience with spreadsheet software (e.g., Excel or Google Sheets) is beneficial. This position is well-suited for individuals looking for a quiet, structured work environment where accuracy and consistency are valued."""}
,{"JobName": "Full-Time Internship – Junior Data Analyst", "JobDescription": """Company: DataSprout Analytics. Type: Internship (6 months), Full-Time. Are you a curious brain who loves spreadsheets more than sweets? DataSprout Analytics is looking for a Junior Data Analyst Intern to join our vibrant Berlin office! This role is a golden ticket into the world of data storytelling, where you'll help us turn raw data into juicy insights for our clients in the e-commerce and fintech industries. You’ll assist with cleaning, analyzing, and visualizing data using tools like Python, SQL, and Tableau (if you don’t know those yet, don’t worry—we love enthusiastic learners!). Our ideal candidate has a strong analytical mindset, good communication skills, and a surprising fondness for Excel formulas. We’ll make sure you’re part of real-world projects from day one. Think of it like data bootcamp, but with free coffee and Friday karaoke. You’ll also get mentorship from seasoned analysts and access to company-wide workshops. This position is perfect for recent grads or students on a gap semester. Proficiency in English is a must, and a pinch of German is a nice bonus. Oh, and we’re dog-friendly—so yes, you’ll meet more than one office dachshund."""}
,{"JobName": "Full-Time Junior Architect", "JobDescription": """Company: Estúdio Horizonte. Type: Full-Time. Calling all architecture graduates who sketch on napkins and dream in CAD! Estúdio Horizonte is hiring a full-time Junior Architect to join our São Paulo-based team focused on sustainable urban housing and community-centric design. You’ll work closely with senior architects on project planning, client presentations, and 3D modeling (we use Revit, Rhino, and AutoCAD). This is a great opportunity to gain exposure to all stages of the architectural process—from concept to construction site. We’re looking for someone passionate, precise, and eager to learn. Fluency in Portuguese is required, and English is a nice-to-have for international projects. You'll thrive here if you're collaborative, detail-obsessed, and up for a design challenge or two. We offer a supportive, creative environment where your ideas will be heard—and your coffee cup will never be empty."""}
,{"JobName": "Full-Time Software Engineer", "JobDescription": """Company: NeonFox Robotics. Type: Full-Time, On-site. Kon'nichiwa, code samurai! NeonFox Robotics is hunting for a backend software engineer to help us build the AI systems that power next-gen robots. Based in the heart of Tokyo, this is your chance to join a cutting-edge startup changing the future of automation. Your role will involve designing scalable APIs, managing cloud infrastructure, and collaborating with robotics engineers to turn code into motion. We work primarily in Go and Python, so experience with either is a major plus. You’ll also need to be comfortable with Docker, RESTful APIs, and cloud platforms like AWS. We're a team of tech dreamers, anime nerds, and coffee aficionados. You’ll be immersed in a culture that balances high-tech innovation with high-quality ramen lunches. Japanese language skills are helpful but not required—we speak code fluently. We offer relocation support, housing assistance, and weekly bento box tastings. If you love the smell of fresh hardware and the hum of server logs, you’ll feel right at home."""}
,{"JobName": "Junior Graphic Designer", "JobDescription": """Company: PixelRoot Creative Studio. We are seeking a creative and detail-oriented Junior Graphic Designer to support our design team in producing compelling visual assets. This role is perfect for someone with a passion for design and an eye for layout, color, and typography. Your responsibilities will include assisting in the creation of digital graphics, social media visuals, marketing materials, and internal branding elements. You’ll work closely with senior designers and stakeholders to ensure brand consistency and high design standards. The ideal candidate is proficient in design software such as Adobe Photoshop, Illustrator, and InDesign. Basic knowledge of Figma or similar tools is a plus. We value creative collaboration and encourage input from all team members. If you're excited about telling stories through visuals and learning from experienced professionals, this could be the right fit for you."""}
,{"JobName": "Marketing Assistant (student position)", "JobDescription": """Company: BrightLane Marketing. We are looking for a motivated and creative student to join our marketing team as a part-time Marketing Assistant. This role is ideal for someone currently pursuing a degree in marketing, communications, business, or a related field. You will support our marketing campaigns, assist with social media management, and help maintain promotional content across various channels. The successful candidate will have a keen interest in digital marketing and be eager to gain hands-on experience in a professional environment. You will work closely with our marketing coordinator and content team to help execute campaign strategies, prepare presentations, and conduct market research. This is a flexible student job designed to fit around your academic schedule. You will be expected to work approximately 10–15 hours per week. We offer a supportive learning environment and the opportunity to build a solid foundation in marketing practices. Experience with tools like Canva, Google Analytics, or social media platforms is a plus but not required. Strong communication skills and attention to detail are essential."""}
,{"JobName": "Part-Time Retail Assistant – Bookstore", "JobDescription": """Company: Paper Trail Books. Type: Part-Time (Weekends + 2 weekdays). Are you a bookworm who also knows how to wield a barcode scanner? Paper Trail Books in Cape Town is on the lookout for a cheerful and reliable part-time assistant who loves literature, lattes, and local authors. Your main tasks will include restocking shelves, helping customers find their next favorite read, organizing cozy book club nights, and occasionally recommending that one book that changed your life (we all have one). Bonus points if you can alphabetize backwards. You’ll be working with a small but mighty team of fellow book lovers who believe a bookstore should feel like a second home. Shifts are flexible, but weekend availability is key—especially during our busy Saturday rush and Sunday poetry afternoons. Fluent English is required, and if you speak Afrikaans or Xhosa, that’s even better. A love of cats is helpful (ours likes to nap on the register)."""}
,{"JobName": "Part-Time Social Media Manager – Café & Bakery", "JobDescription": """Company: Butter & Crumbs. Type: Part-Time (15–20 hrs/week). Love lattes, filters, and food puns? Butter & Crumbs, a cozy artisan café in downtown Toronto, is looking for a part-time social media whiz to help us make our pastries go viral. You’ll create weekly content (Reels, posts, stories), manage our online community, collaborate with baristas for trending TikToks, and maybe even help name our seasonal scones. Creativity and a good eye are more important than formal marketing degrees. Basic photo and video editing skills are required (we love CapCut and Canva around here), and you should be comfy with Instagram, TikTok, and Facebook. We encourage playfulness and personality in our brand—our fans expect it! Free croissants with every shift, plus the opportunity to shape the café’s voice. Ideal for students or freelancers with flexible hours and a flair for storytelling."""}
,{"JobName": "Remote Freelance Graphic Designer ", "JobDescription": """Company: PixelPeach Creative Studio. Type: Freelance, Remote. Calling all pixel magicians! PixelPeach is looking for a freelance graphic designer who’s got a keen eye for color, a killer aesthetic, and the ability to create scroll-stopping visuals for clients around the globe. You’ll be working on branding kits, social media assets, digital ads, and the occasional offbeat zine cover. Projects are varied and fast-paced, so being comfortable with Adobe CC (especially Illustrator, Photoshop, and InDesign) is a must. If you’re handy with Figma, that’s a peachy bonus. We love quirky portfolios, bold ideas, and designers who aren't afraid of a good moodboard. While this role is freelance, we work collaboratively through Slack and Notion, and we expect responsiveness during core collaboration hours (GMT+0 to GMT+3 preferred). Perfect for creative spirits who thrive in flexible environments and have a reliable Wi-Fi connection (coffee shop creatives welcome). Payment is project-based and competitive."""}
,{"JobName": "Retail Sales Associate", "JobDescription": """Company: NovaStyle Retail. We are hiring a customer-focused and enthusiastic Retail Sales Associate to join our store team. This role is ideal for someone with strong interpersonal skills who enjoys engaging with customers and delivering a positive shopping experience. As a Retail Sales Associate, you will be responsible for assisting customers, handling transactions, restocking merchandise, and maintaining store cleanliness. You’ll be expected to stay informed about current promotions and product knowledge in order to offer helpful recommendations. The ideal candidate is friendly, reliable, and able to work independently in a fast-paced environment. Weekend and evening shifts may be required, depending on store needs. No prior experience is necessary—just a great attitude and willingness to learn. We provide full training and opportunities for advancement."""}
,{"JobName": "Software Engneering Intern", "JobDescription": """Company: CodeNova Technologies. We are seeking a driven and curious software engineering intern to join our development team. As an intern, you will gain valuable experience working on real-world software projects under the guidance of experienced engineers. This is an ideal opportunity for students in computer science, software engineering, or related fields looking to apply their academic knowledge in a hands-on setting. You will contribute to coding tasks, assist in debugging and testing, and participate in team standups and sprint planning. The team uses agile methodologies, and you'll be exposed to the full software development lifecycle. We value creative thinkers who are not afraid to ask questions and challenge assumptions. Your internship will include one-on-one mentorship, feedback sessions, and opportunities to explore different aspects of the tech stack. Proficiency in languages like Python, JavaScript, or Java is preferred. Familiarity with Git and basic knowledge of version control is expected."""}
,{"JobName": "Summer Internship – Environmental Research Assistant ", "JobDescription": """Company: NordicNature Institute. Type: Internship (3 months, Summer). Are you passionate about the planet and enjoy fieldwork more than filing? NordicNature is looking for a spirited intern to join our summer research program studying freshwater ecosystems in and around Oslo. You’ll assist senior researchers in data collection, lab analysis, and species cataloging. The role includes a healthy mix of time outdoors (waders provided!) and office tasks like digitizing records and prepping reports. A background in biology or environmental science is preferred, but passion trumps pedigree. Expect early mornings, breathtaking landscapes, and the occasional unexpected moose sighting. We foster a supportive and inclusive environment where learning by doing is the motto. Fluency in English is required, and basic Norwegian will help with local signage and friendly chats with hikers. A strong back, waterproof notebook, and enthusiasm for bugs are essential."""}
,{"JobName": "Tech Job", "JobDescription": """Job Description
Position Title: Senior Full-Stack Software Engineer

Location: Boston, MA (Hybrid)

Company: BrightNova Technologies

About the Role:
BrightNova Technologies is seeking a highly skilled and experienced Senior Full-Stack Software Engineer to lead the development and maintenance of our multi-tenant SaaS platform. The ideal candidate will have extensive experience in full-stack development, cloud infrastructure, microservices architecture, and DevOps practices.

Responsibilities:
Lead design and development of scalable web applications using React.js, Node.js, and Spring Boot.

Architect microservices and implement RESTful and GraphQL APIs.

Manage cloud infrastructure on AWS (EC2, S3, Lambda, RDS) and optimize for cost and performance.

Build and maintain CI/CD pipelines using Jenkins, GitHub Actions, and Docker.

Conduct code reviews, mentor junior developers, and enforce coding standards.

Collaborate with cross-functional teams (Product, QA, Design) in Agile/Scrum environment.

Integrate third-party APIs for payments, analytics, and authentication.

Optimize backend performance, database queries (PostgreSQL, MongoDB), and caching mechanisms (Redis).

Requirements:
8+ years of full-stack development experience.

Strong proficiency in React.js, Angular, Vue.js, TypeScript, HTML5, CSS3.

Deep knowledge of backend development using Node.js, Python (Django/Flask), Java (Spring Boot).

Expertise with AWS services and cloud infrastructure management.

Experience with Docker, Kubernetes, and CI/CD pipelines.

Strong understanding of microservices architecture and RESTful/GraphQL APIs.

Experience in database management (PostgreSQL, MongoDB, MySQL, Redis).

Familiarity with Agile methodologies, Jira, and Confluence.

Excellent problem-solving and communication skills.

Bachelor's degree in Computer Science or related field.

AWS Certified Solutions Architect – Associate and CKAD certification.

Preferred:
Experience mentoring junior engineers.

Scrum Master Certification.

Fluent in Spanish.

Benefits:
Competitive salary and performance bonuses

Hybrid work model with flexible hours

Health, dental, and vision insurance

401(k) with company match

Professional development budget

Generous PTO and paid holidays

"""}]

def process_text_to_json(pdf_text, country):
  number_of_jobs = get_country_value(country)
  found_jobs = []
  for _ in range(number_of_jobs):
    random_job = random.choice(JobDescriptions)
    job_name = random_job["JobName"]
    job_description = random_job["JobDescription"]
    
    # model = Doc2Vec.load('cv_job_maching.model')
    # try:
    #   model = Doc2Vec.load('cv_job_maching.model')
    # except FileNotFoundError:
    #   return [{"name": "Error", "percentage": 0, "color": "hsl(0, 70%, 60%)"}]

    # v1 = model.infer_vector(preprocess_text(pdf_text).split())
    # v2 = model.infer_vector(preprocess_text(job_description).split())
    # similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
    # percentage = round(similarity, 2)

    llm_ranker_responce = LLM_ranker(pdf_text, job_description)

    percentage = float(llm_ranker_responce) + float(random.random() * 15)

    if(percentage > 100):
        percentage = 100

    percentage = round(percentage, 2)

    
    found_jobs.append({
      "name": job_name,
      "percentage": percentage,
      "color": f"hsl({percentage_to_color(percentage)}, 70%, 60%)"
    })
  found_jobs.sort(key=lambda x: x["percentage"], reverse=True)
  return found_jobs


HTML = """
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <title>Upload PDF with Country Selection</title>
    <style>
      body {
        background: linear-gradient(
          to right,
          hsl(210, 17%, 8%),
          hsl(191, 89%, 18%)
        );
        font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif;
        display: flex;
        justify-content: center;
        align-items: center;
        flex-direction: column;
        padding: 50px;
        color: white;
      }

      h2 {
        background: linear-gradient(to right, #007bff, #00bcd4);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        font-size: 2em;
        margin-bottom: 10px;
      }

      img.logo {
        width: 200px;
        margin-bottom: 20px;
      }

      form {
        backdrop-filter: blur(16px) saturate(180%);
        -webkit-backdrop-filter: blur(16px) saturate(180%);
        background-color: rgba(255, 255, 255, 0.1);
        border: 1px solid rgba(255, 255, 255, 0.3);
        padding: 30px;
        border-radius: 15px;
        box-shadow: 0 8px 20px rgba(0, 0, 0, 0.1);
        display: flex;
        flex-direction: column;
        gap: 15px;
        width: 100%;
        max-width: 400px;
        color: white;
      }

      label {
        font-weight: bold;
      }

      option{
        color: #090305
      }

      input[type="file"],
      select {
        border: 1px solid #ccc;
        padding: 10px;
        border-radius: 10px;
        background: rgba(255, 255, 255, 0.2);
        color: white;
      }

      input[type="submit"] {
        background: linear-gradient(to right, #00bcd4, #007bff);
        color: white;
        border: none;
        padding: 10px 20px;
        border-radius: 10px;
        cursor: pointer;
        transition: background 0.3s ease;
      }

      input[type="submit"]:hover {
        background: linear-gradient(to right, #007bff, #0056b3);
      }

      pre {
        background: #f5f5f5;
        padding: 20px;
        border-radius: 10px;
        margin-top: 30px;
        white-space: pre-wrap;
        word-wrap: break-word;
        max-width: 800px;
        color: black;
      }

      .info {
        margin-top: 20px;
        font-size: 1.1em;
      }
    </style>
  </head>
  <body>
    <img
      src="{{ url_for('static', filename='img/WorkShip2.png') }}"
      alt="Logo"
      class="logo"
    />
    <h2>Upload a PDF</h2>
    <form method="post" enctype="multipart/form-data">
      <label for="file">Select PDF:</label>
      <input type="file" name="file" accept=".pdf" required />

      <label for="country">Select Country:</label>
      <select name="country" required>
        <option value="Europe">🇪🇺 Europe</option>
        <option value="Austria">🇦🇹 Austria</option>
        <option value="Belgium">🇧🇪 Belgium</option>
        <option value="Bulgaria">🇧🇬 Bulgaria</option>
        <option value="Croatia">🇭🇷 Croatia</option>
        <option value="Cyprus">🇨🇾 Cyprus</option>
        <option value="Czech Republic">🇨🇿 Czech Republic</option>
        <option value="Denmark">🇩🇰 Denmark</option>
        <option value="Estonia">🇪🇪 Estonia</option>
        <option value="Finland">🇫🇮 Finland</option>
        <option value="France">🇫🇷 France</option>
        <option value="Germany">🇩🇪 Germany</option>
        <option value="Greece">🇬🇷 Greece</option>
        <option value="Hungary">🇭🇺 Hungary</option>
        <option value="Ireland">🇮🇪 Ireland</option>
        <option value="Italy">🇮🇹 Italy</option>
        <option value="Latvia">🇱🇻 Latvia</option>
        <option value="Lithuania">🇱🇹 Lithuania</option>
        <option value="Luxembourg">🇱🇺 Luxembourg</option>
        <option value="Malta">🇲🇹 Malta</option>
        <option value="Netherlands">🇳🇱 Netherlands</option>
        <option value="Poland">🇵🇱 Poland</option>
        <option value="Portugal">🇵🇹 Portugal</option>
        <option value="Romania">🇷🇴 Romania</option>
        <option value="Slovakia">🇸🇰 Slovakia</option>
        <option value="Slovenia">🇸🇮 Slovenia</option>
        <option value="Spain">🇪🇸 Spain</option>
        <option value="Sweden">🇸🇪 Sweden</option>
      </select>
      <input type="submit" value="Upload & Analyze CV" />
    </form>

    {% if country %}
    <div class="info">
      📍 Selected Country:
      <strong>{{ country }}</strong>
    </div>
    {% endif %} {% if text %} {% if results %}
    <h3 style="margin-top: 40px">Analysis Results:</h3>
    <div
      style="
        display: flex;
        flex-direction: column;
        gap: 10px;
        margin-top: 10px;
        width: 90%;
        max-width: 800px;
      "
    >
      {% for item in results %}
      <div
        style="
          display: flex;
          justify-content: space-between;
          align-items: center;
          background: rgba(255, 255, 255, 0.1);
          padding: 10px 15px;
          border-radius: 8px;
        "
      >
        <strong>{{ item.name }}</strong>
        <div
          style="
            width: 20%;
            min-width: 64px;
            background: #eee;
            border-radius: 6px;
            overflow: hidden;
          "
        >
          <div
            style="width: {{ item.percentage }}%; background-color: {{ item.color }}; padding: 5px 0; text-align: center; color: black;"
          >
            {{ item.percentage }}%
          </div>
        </div>
      </div>
      {% endfor %}
    </div>
    {% endif %} {% endif %}
  </body>
</html>

"""


app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def upload_pdf():
    text = None
    country = None
    results = None

    if request.method == "POST":
        country = request.form.get("country")
        file = request.files["file"]
        if file and file.filename.endswith(".pdf"):
            doc = fitz.open(stream=file.read(), filetype="pdf")
            text = "\n".join(page.get_text() for page in doc)
            results = process_text_to_json(text, country)

    print(f"country: {country}\ntext: {results}")
    return render_template_string(HTML, text=text, country=country, results=results)


if __name__ == '__main__':
    url = 'http://127.0.0.1:5000'
    threading.Timer(1, lambda: webbrowser.open(url)).start()
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)
